# Feature Selection


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.chdir("..")

In [2]:
from IPython.core.debugger import Tracer
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
from scipy.signal import welch
from IPython.core.debugger import Tracer
from sklearn_pandas import DataFrameMapper, cross_val_score
from p300.feature_extraction import BaseTransformer

In [3]:
hdf = pd.HDFStore("output/instances.h5")


hdf.keys()

['/subjects/s10229001',
 '/subjects/s10444001',
 '/subjects/s10729001',
 '/subjects/s10882001',
 '/subjects/s10924001',
 '/subjects/s11551001',
 '/subjects/s11627001',
 '/subjects/s11632001',
 '/subjects/s11693001',
 '/subjects/s12137001',
 '/subjects/s12168001',
 '/subjects/s12521001',
 '/subjects/s12702001',
 '/subjects/s12900001',
 '/subjects/s13235001',
 '/subjects/s13252001',
 '/subjects/s13431001',
 '/subjects/s13640002',
 '/subjects/s13863001',
 '/subjects/s14023001',
 '/subjects/s1414001',
 '/subjects/s1491001',
 '/subjects/s14998001',
 '/subjects/s15362001',
 '/subjects/s15424001',
 '/subjects/s15641001',
 '/subjects/s16003001',
 '/subjects/s1609001',
 '/subjects/s16266001',
 '/subjects/s1635001',
 '/subjects/s16637001',
 '/subjects/s16683001',
 '/subjects/s16779001',
 '/subjects/s16893001',
 '/subjects/s16943001',
 '/subjects/s17005001',
 '/subjects/s17435001',
 '/subjects/s17436001',
 '/subjects/s17576001',
 '/subjects/s17674001',
 '/subjects/s17962001',
 '/subjects/s1804600

Veamos cuántos sujetos tenemos

Tomemos uno: 29164001

In [4]:
from sklearn.model_selection import cross_val_score

non_features = ['id', 'array_path', 'ch_names', 'event_time', 'event_type', 'index',
       'sfreq', 'subject_id', 'target']

def get_data_from_subject(hdf, key):
    df = hdf[key]
    X = df[df.columns.difference(non_features)]
    y = df.target.as_matrix()
    
    return X.as_matrix(), y, X.columns

X, y, names = get_data_from_subject(hdf, '/subjects/s5224001')

X.shape

(1980, 112)

In [5]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE

clf = LinearDiscriminantAnalysis()

selector = RFE(clf, 40, step=1)

In [6]:
selector.fit(X, y)

RFE(estimator=LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001),
  n_features_to_select=40, step=1, verbose=0)

In [7]:
names[selector.ranking_[:40]]

Index(['subsample___AF3_2', 'subsample___F3_1', 'subsample___F4_1',
       'subsample___FC5_7', 'subsample___F7_1', 'subsample___F7_0',
       'subsample___FC6_7', 'subsample___F4_6', 'subsample___AF4_1',
       'subsample___FC5_3', 'subsample___F4_0', 'subsample___F8_2',
       'subsample___O2_0', 'subsample___F4_4', 'subsample___F7_4',
       'subsample___AF4_6', 'subsample___AF3_1', 'subsample___F3_2',
       'subsample___F3_3', 'subsample___F3_4', 'subsample___AF3_1',
       'subsample___AF3_1', 'subsample___O1_3', 'subsample___F3_7',
       'subsample___AF3_1', 'subsample___F7_3', 'subsample___F7_2',
       'subsample___AF3_1', 'subsample___O1_4', 'subsample___AF3_1',
       'subsample___FC5_5', 'subsample___F3_6', 'subsample___FC6_3',
       'subsample___O1_0', 'subsample___O1_6', 'subsample___O1_7',
       'subsample___FC6_4', 'subsample___FC6_5', 'subsample___F8_7',
       'subsample___F8_6'],
      dtype='object')

In [16]:
selector = RFE(clf, 20, step=1)
clf = LinearDiscriminantAnalysis()
X, y, names = get_data_from_subject(hdf, '/subjects/s26333001')

X.shape

(1980, 112)

In [17]:
selector.fit(X, y)

names[selector.ranking_[:20]]

Index(['subsample___AF3_1', 'subsample___F4_2', 'subsample___FC6_0',
       'subsample___FC5_7', 'subsample___O1_1', 'subsample___P8_5',
       'subsample___FC5_6', 'subsample___FC5_4', 'subsample___AF3_1',
       'subsample___AF3_1', 'subsample___AF3_4', 'subsample___FC5_0',
       'subsample___O1_5', 'subsample___FC5_1', 'subsample___O2_1',
       'subsample___F8_7', 'subsample___AF3_1', 'subsample___AF3_1',
       'subsample___AF3_3', 'subsample___AF3_1'],
      dtype='object')

Al parecer, los sensores más importantes son los frontales.

In [21]:
from sklearn.model_selection import cross_val_score

cross_val_score(selector, X, y, scoring='roc_auc', cv=10).mean()

0.5757208448117539